In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_selection import RFE
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.feature_selection import SelectPercentile, SelectKBest, SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score

from sklearn import metrics
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import seaborn as sns

import scikitplot as skplt

import matplotlib.pyplot as plt
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def distribucion(attr,datos):
    plt.figure()
    sns.distplot(datos[attr])
    
def relevancia(clase,attr,datos):
    plt.figure()
    gs = datos.groupby(clase)
    for x in datos[clase].unique():
        plt.legend()
        sns.distplot(gs.get_group(x)[attr], label=x)
        
def cuenta(attr,datos):
    plt.figure()
    sns.countplot(y=attr,data=datos)
    
def muestra_relevancias(nombres, relevancias):
    '''Función para mostrar una gráfica con el grado de relevancia de cada atributo
    
    Recibe:
       - nombres: nombres de todos los atributos
       - relevancias: de cada atributo, calculadas mediante alguna técnica
    '''
    plt.figure(figsize=(len(nombres)/4,5))
    serie = pd.Series(relevancias, index=nombres)
    serie.plot(kind='bar')
    
def muestra_seleccion(nombres, mascara):
    '''Función para mostrar los nombres de los atributos seleccionados a partir de una máscara de booleanos
    
    Recibe:
       - nombres: nombres de todos los atributos
       - mascara: lista de booleanos que determina si un atributo se selecciona, o no
    '''    
    seleccionados = [n for n,m in zip(nombres, mascara) if m]
    print(len(seleccionados), seleccionados)
    

def muestra_evolucion_un_parametro(grid, parametro):
    ''' Muestra la evolución de los resultados de gridsearch en función de un parámetro 
    '''
    # Volcamos los resultados en un dataframe y seleccionamos la columna del parámetro a visualizar
    valores_params = pd.DataFrame(list(grid.cv_results_['params']))[parametro]
    
    # Creamos una serie con los resultados y usando como índice los valores del parámetro
    resultados = pd.Series(grid.cv_results_['mean_test_score'], index=valores_params)
    resultados.plot()
    

def muestra_evolucion_dos_parametros(grid, parametros):
    ''' Muestra la evolución de los resultados de gridsearch en función de un parámetro 
    '''
    # Volcamos los resultados en un dataframe y seleccionamos las columnas de los parámetros a visualizar
    valores_params = pd.DataFrame(list(grid.cv_results_['params']))[parametros]
    
    # Creamos un DataFrame con tres columnas: los resultados y los valores de los dos paráemtros
    resultados = pd.DataFrame(list(grid.cv_results_['mean_test_score']), columns=['resultados'])
    resultados = pd.concat([resultados, valores_params], axis=1)
    
    # Creamos una tabla pivot a partir de losn resultados y la visualizamos
    tabla = resultados.pivot_table(index=parametros[0], columns=parametros[1], values='resultados', fill_value=0)
    sns.heatmap(tabla, annot=True, fmt='.4f', cmap='Greens')
    plt.show()

In [6]:
m_cols = ['id', 'year', 'title']
movies = pd.read_csv("data/movie_titles.csv",sep=',', names=m_cols, encoding='latin-1').set_index('id')

In [7]:
movies.head()

,year,title
id,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW
